In [2]:
import numpy as np
import pandas as pd
import smtplib
import torch
import torch.nn as nn
import models

from email.message import EmailMessage
from matplotlib.pylab import plt
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

In [2]:
import importlib
importlib.reload(models)

<module 'models' from '/home/mma6789/Stuff/Studies/sem3/ms_project/models.py'>

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')

Using device: cuda
Quadro RTX 6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [16]:
base_dir = '/home/mma6789/Stuff/Studies/sem3/ms_project' #@param {type: 'string'}

dataset = 'US-101' #@param ['I-80', 'US-101']
t_o = 5000 #@param [3000, 4000, 5000] -> observation horizon
t_p = 2000 #@param [1500, 2000, 2500] -> prediction horizon

method = 'LSTMHMM' #@param ['LSTM', 'HMM', 'LSTMHMM']

timesteps = t_o // 100
variables = 5

In [5]:
## Load class weights
class_weights = np.load(f'{base_dir}/data/processed/{dataset}/{t_o}_{t_p}_class_weights.npy')
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

class_weights

tensor([14.3963,  0.3458, 25.8265], device='cuda:0')

In [6]:
## Load data splits
train = np.load(f'{base_dir}/data/processed/{dataset}/{t_o}_{t_p}_train.npy')
valid = np.load(f'{base_dir}/data/processed/{dataset}/{t_o}_{t_p}_valid.npy')

train_X, train_y = np.split(train, [-3], axis=1)
valid_X, valid_y = np.split(valid, [-3], axis=1)

temp = np.empty((len(train_X), timesteps, variables))
for i in range(len(train_X)):
    temp[i] = np.array(np.split(train_X[i], timesteps))
train_X = temp

temp = np.empty((len(valid_X), timesteps, variables))
for i in range(len(valid_X)):
    temp[i] = np.array(np.split(valid_X[i], timesteps))
valid_X = temp

In [7]:
print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape)

(1796828, 50, 5) (1796828, 3) (385035, 50, 5) (385035, 3) (385035, 50, 5) (385035, 3)


In [8]:
train_X = torch.tensor(train_X).float()
train_y = torch.tensor(train_y).float()
valid_X = torch.tensor(valid_X).float()
valid_y = torch.tensor(valid_y).float()
test_X = torch.tensor(test_X).float()
test_y = torch.tensor(test_y).float()

In [9]:
class LSTMDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]
    
train_dataset = LSTMDataset(train_X, train_y)
valid_dataset = LSTMDataset(valid_X, valid_y)
test_dataset = LSTMDataset(test_X, test_y)

In [10]:
batch_size = 192

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
def train_epoch():
    model.train(True)
    running_loss = 0.0
    y_true = []
    y_pred = []
    
    for batch_index, batch in enumerate(tqdm(train_loader, desc=f'Epoch {epoch + 1} train')):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()
        
        y_true += torch.argmax(y_batch, dim=1).flatten().tolist()
        y_pred += torch.argmax(output, dim=1).cpu().detach().numpy().tolist()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_batch_loss = running_loss / len(train_loader)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    matrix = confusion_matrix(y_true, y_pred)
    class_acc = matrix.diagonal() / matrix.sum(axis=1)
    
    train_loss.append(avg_batch_loss)
    train_acc.append(acc)
    train_f1.append(f1)
    train_class_acc.append(class_acc)
    
    print('Train epoch results:')
    print(f'Loss: {avg_batch_loss}')
    print(f'Acc: {acc}')
    print(f'F1: {f1}')
    print(f'Class Acc: {class_acc}')

In [12]:
def validate_epoch():
    model.train(False)
    running_loss = 0.0
    y_true = []
    y_pred = []
    
    for batch_index, batch in enumerate(tqdm(valid_loader, desc=f'Epoch {epoch + 1} valid')):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()
            
        y_true += torch.argmax(y_batch, dim=1).flatten().tolist()
        y_pred += torch.argmax(output, dim=1).cpu().detach().numpy().tolist()
            
    avg_batch_loss = running_loss / len(valid_loader)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    matrix = confusion_matrix(y_true, y_pred)
    class_acc = matrix.diagonal() / matrix.sum(axis=1)
    
    valid_loss.append(avg_batch_loss)
    valid_acc.append(acc)
    valid_f1.append(f1)
    valid_class_acc.append(class_acc)
    
    print('Valid epoch results:')
    print(f'Loss: {avg_batch_loss}')
    print(f'Acc: {acc}')
    print(f'F1: {f1}')
    print(f'Class Acc: {class_acc}')
    print('***************************************************')

In [17]:
## Initialize model
if method == 'LSTM':
    input_size = variables
    hidden_size = 150
    num_layers = 1
    output_size = 3

    model = models.Three_LSTM(input_size, hidden_size, num_layers, output_size)
    model.to(device)
elif method == 'HMM':
    input_size = variables
    hidden_states = 30
    output_size = 3
    
    model = models.Three_HMM(input_size, hidden_states, output_size)
    model.to(device)
elif method == 'LSTMHMM':
    input_size = variables
    hidden_size = 150
    hidden_states = 30
    output_size = 3
    
    model = models.Three_LSTMHMM(input_size, hidden_size, hidden_states, output_size)
    model.to(device)

model

Three_LSTMHMM(
  (lcl_model): LSTMHMM(
    (transition_model): TransitionModel()
    (emission_model): LSTMEmissionModel(
      (lstm): LSTM(5, 150, batch_first=True)
      (linear): Linear(in_features=150, out_features=30, bias=True)
    )
  )
  (lk_model): LSTMHMM(
    (transition_model): TransitionModel()
    (emission_model): LSTMEmissionModel(
      (lstm): LSTM(5, 150, batch_first=True)
      (linear): Linear(in_features=150, out_features=30, bias=True)
    )
  )
  (lcr_model): LSTMHMM(
    (transition_model): TransitionModel()
    (emission_model): LSTMEmissionModel(
      (lstm): LSTM(5, 150, batch_first=True)
      (linear): Linear(in_features=150, out_features=30, bias=True)
    )
  )
)

In [18]:
## Train model
learning_rate = 0.0002
num_epochs = 140

loss_function = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loss = []
train_acc = []
train_f1 = []
train_class_acc = []
valid_loss = []
valid_acc = []
valid_f1 = []
valid_class_acc = []

for epoch in tqdm(range(num_epochs), desc='Training progress'):
    train_epoch()
    validate_epoch()

Training progress:   0%|          | 0/140 [00:00<?, ?it/s]

Epoch 1 train:   0%|          | 0/9359 [00:00<?, ?it/s]

Train epoch results:
Loss: 0.9124986920260966
Acc: 0.6283044342585935
F1: 0.7454317783609521
Class Acc: [0.57516387 0.63172877 0.46867351]


Epoch 1 valid:   0%|          | 0/2006 [00:00<?, ?it/s]

Valid epoch results:
Loss: 0.8855455223459784
Acc: 0.7929071383121015
F1: 0.85776625678017
Class Acc: [0.48700707 0.80531054 0.40568319]
***************************************************


Epoch 2 train:   0%|          | 0/9359 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [27]:
## Save model to file
torch.save(model, f'{base_dir}/models/{method}/{dataset}/model_{t_o}_{t_p}_{learning_rate}.pt')

In [28]:
## Write metrics to file
metrics = pd.DataFrame(data={
    'train_loss': train_loss, 'train_acc': train_acc, 'train_f1': train_f1, 'train_class_acc': train_class_acc,
    'valid_loss': valid_loss, 'valid_acc': valid_acc, 'valid_f1': valid_f1, 'valid_class_acc': valid_class_acc,
})

metrics.to_json(f'{base_dir}/metrics/{method}/{dataset}/training_{t_o}_{t_p}_{learning_rate}.json')

In [5]:
msg = EmailMessage()

msg.set_content(metrics.to_string())
msg['Subject'] = f'Testing metrics of {method} on {dataset} with {learning_rate}'

msg['From'] = 'mma6789@psu.edu'
msg['To'] = 'mahmad97taha@gmail.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

(221, b'2.0.0 h3shannon.cs.hbg.psu.edu closing connection')